### create_edp_df debug

In [1]:
import sys
import os
import numpy as np

import pandas as pd
idx = pd.IndexSlice
pd.options.display.max_rows = 30

In [32]:
# baseDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/RegionalStudy/Codes/woodSDPA'
# ID = 'site0_s1_96x48_Stucco_GWB_Normal_Vs32'
# # ID = 's3_96x48_Moderate_HWS_GWB_Light'
# baselineID = 's1_96x48'
# pfa = pd.read_csv(os.path.join(baseDir, *['Results', 'HiFi_FMA', ID, 'EDP_data', 'PFA.csv']), header=None)
# sdr = pd.read_csv(os.path.join(baseDir, *['Results', 'HiFi_FMA', ID, 'EDP_data', 'SDR.csv']), header=None)
# rdr = pd.read_csv(os.path.join(baseDir, *['Results', 'HiFi_FMA', ID, 'EDP_data', 'RDR.csv']), header=None)

In [33]:
# col_names_drift = ['IM', 'Direction', 'GM_ID', 'Story_1', 'Story_2', 'Story_3', 'Story_4']
# col_names_pfa = ['IM', 'Direction', 'GM_ID', 'Story_0', 'Story_1', 'Story_2', 'Story_3', 'Story_4']
# sdr = sdr.rename(columns= dict(zip(sdr.columns, col_names_drift)))
# pfa = pfa.rename(columns = dict(zip(pfa.columns, col_names_pfa)))
# rdr = sdr.rename(columns= dict(zip(rdr.columns, col_names_drift)))
# edp_list = [sdr, pfa, rdr]
# edp_name = ['PID', 'PFA', 'RID']

In [34]:
# edp_list[i]['Story_1'].values

array([1.43883e+10, 5.23476e-02, 4.76297e-04, 1.08193e-03, 2.29206e-02,
       1.54685e-02, 1.80908e-03, 4.12214e-03, 2.87738e-03, 2.85749e-03,
       1.35693e-03, 7.19057e-03, 3.23691e-04, 7.82430e-04, 8.30637e-03,
       2.59372e-03, 1.93672e-03, 1.97810e-02, 1.84350e-03, 1.59206e-03,
       1.10234e+11, 3.46217e+07, 1.20962e-03, 1.09057e-03, 8.25171e-03,
       3.49697e-03, 3.36179e-03, 3.50139e-02, 2.27304e-03, 2.67317e-03,
       1.94775e-03, 9.97606e-03, 7.32158e-04, 1.22613e-03, 4.40519e-03,
       2.66203e-03, 1.74809e-03, 1.77023e-02, 2.16994e-03, 3.05321e-03])

In [35]:
# i=0
# k=1
# level=1
# collapse_trigger = 1
# temp_edp_df = edp_list[i].loc[(edp_list[i]['Direction']==k) & 
#                                               (edp_list[i]['Story_%s'%level] > collapse_trigger), 
#                                                'Story_%s'%level] = edp_list[i]['Story_%s'%level].min()

In [37]:
# edp_list[i].loc[(edp_list[i]['Direction']==k) & 
#                                               (edp_list[i]['Story_%s'%level] > collapse_trigger), 
#                                                'Story_%s'%level] = edp_list[i]['Story_%s'%level].quantile(q=0.9)

In [39]:
# edp_list[i].loc[(edp_list[i]['Direction']==k+1) & 
#                                               (edp_list[i]['Story_%s'%level] > collapse_trigger), 
#                                                'Story_%s'%level] = edp_list[i]['Story_%s'%level].quantile(q=0.9)

In [40]:
# edp_list[i]['Story_1'].values

array([0.00032369, 0.0523476 , 0.0004763 , 0.00108193, 0.0229206 ,
       0.0154685 , 0.00180908, 0.00412214, 0.00287738, 0.00285749,
       0.00135693, 0.00719057, 0.00032369, 0.00078243, 0.00830637,
       0.00259372, 0.00193672, 0.019781  , 0.0018435 , 0.00159206,
       0.02412993, 0.02412993, 0.00120962, 0.00109057, 0.00825171,
       0.00349697, 0.00336179, 0.0350139 , 0.00227304, 0.00267317,
       0.00194775, 0.00997606, 0.00073216, 0.00122613, 0.00440519,
       0.00266203, 0.00174809, 0.0177023 , 0.00216994, 0.00305321])

In [11]:
def compile_demand_data(edp_list, edp_name):

    index_name = []
    median = []
    log_std = []
    for i in range(len(edp_list)):
        baseline_num = 3 if edp_name[i]=='PFA' else 2
        edp = edp_list[i]
        ## filter out collpase casese
        collapse_limit = 3 if edp_name[i]=='PFA' else 1
        
        for stripe in edp[0].unique():
            churn_df = edp[edp[0]==stripe]
            story_name = 0 
            for story in churn_df.columns:
                for uniq_dir in edp[1].unique():
                    edp = edp_list[i][(edp_list[i][story]<= collapse_limit)]
                    if story >=3:
                        story_name = story - baseline_num
                        index_name.append('%s-%s-%s-%s'%(stripe, edp_name_mapping[edp_name[i]], story_name, uniq_dir))
                        median.append(edp[(edp[0]==stripe) & (edp[1]==uniq_dir)][story].median())
                        log_std.append(edp[(edp[0]==stripe) & (edp[1]==uniq_dir)][story].std())
    d = {
        'idx': index_name, 
        'Theta_0': median, 
        'Theta_1': log_std
    }
    df_edp = pd.DataFrame(d)
    df_edp = df_edp.set_index('idx')

    df_edp = convert_to_MultiIndex(df_edp, axis=0)
    df_edp.index.names = ['stripe','type','loc','dir']
    df_edp.insert(0, 'Family',"")
    df_edp['Family'] = 'lognormal'
    return df_edp


0.000323691

# Setting up the environment


In [2]:
# import helpful packages for numerical analysis
import sys
import os
import numpy as np

import pandas as pd
idx = pd.IndexSlice
pd.options.display.max_rows = 30

import pprint


from pelicun_3_1.base import convert_to_MultiIndex
from pelicun_3_1.assessment import Assessment

from Run_pelicun_v3p1 import compile_demand_data, pelicun_assessment1

%load_ext autoreload
%autoreload 2

In [6]:
# baseDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/SurrogateModeling/DGP/woodSDA'
baseDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/RegionalStudy/woodSDPA'

## Data Pre-processing

In [28]:
# ID = 's4_96x48_High_Stucco_Plaster_Normal'
# baselineID = 's4_96x48'

# pfa = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_four_story', ID, 'EDP_data', 'PFA.csv']), header=None)
# sdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_four_story', ID, 'EDP_data', 'SDR.csv']), header=None)
# rdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_four_story', ID, 'EDP_data', 'RDR.csv']), header=None)

# ID = 's1_40x30_High_Stucco_GWB_Heavy'
# # ID = 's3_96x48_Moderate_HWS_GWB_Light'
# baselineID = 's1_40x30'
# pfa = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_N_20', ID, 'EDP_data', 'PFA.csv']), header=None)
# sdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_N_20', ID, 'EDP_data', 'SDR.csv']), header=None)
# rdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_N_20', ID, 'EDP_data', 'RDR.csv']), header=None)
ID = 's3_96x48_Stucco_Plaster_Light_Vs129'
# ID = 's2_40x30_Stucco_GWB_Normal_Vs20'
split_str = ID.split('_')
baselineID = '_'.join([split_str[0], split_str[1]])
pfa = pd.read_csv(os.path.join(baseDir, *['Results', 'EA-VaFi', ID, 'EDP_data', 'PFA.csv']), header=None)
sdr = pd.read_csv(os.path.join(baseDir, *['Results', 'EA-VaFi', ID, 'EDP_data', 'SDR.csv']), header=None)
rdr = pd.read_csv(os.path.join(baseDir, *['Results', 'EA-VaFi', ID, 'EDP_data', 'RDR.csv']), header=None)

In [70]:
import pandas as pd

# Create a sample DataFrame
data = {
    'ID': [1, 2, 3],
    'Name': ['Alice', 'Bob', 'Charlie'],
    'Test1': [90, 88, 76],
    'Test2': [85, 92, 78],
    'Test3': [88, 90, 82]
}

df = pd.DataFrame(data)

# Melt the DataFrame into a long format
long_df = pd.melt(df, id_vars=['ID', 'Name'], var_name='Test', value_name='Score')

# Display the long DataFrame
print(long_df)


   ID     Name   Test  Score
0   1    Alice  Test1     90
1   2      Bob  Test1     88
2   3  Charlie  Test1     76
3   1    Alice  Test2     85
4   2      Bob  Test2     92
5   3  Charlie  Test2     78
6   1    Alice  Test3     88
7   2      Bob  Test3     90
8   3  Charlie  Test3     82


In [29]:
edp_name_mapping = {'SDR':'PID',
                    'PFA':'PFA', 
                    'RDR':'RID'}
raw_demands = compile_demand_data([sdr, pfa], ['SDR', 'PFA'])


print(raw_demands.head(n=3))
print(raw_demands.tail(n=3))

Family   Theta_0   Theta_1
stripe type loc dir                               
1      PID  1   1    lognormal  0.000248  0.000207
                2    lognormal  0.000332  0.000233
            2   1    lognormal  0.000164  0.000130
                2    lognormal  0.000224  0.000083
2      PID  1   1    lognormal  0.000382  0.000379
                2    lognormal  0.000542  0.000276
            2   1    lognormal  0.000243  0.000197
                2    lognormal  0.000297  0.000141
3      PID  1   1    lognormal  0.000446  0.000272
                2    lognormal  0.000687  0.000485
            2   1    lognormal  0.000273  0.000175
                2    lognormal  0.000361  0.000224
4      PID  1   1    lognormal  0.000691  0.000259
                2    lognormal  0.000911  0.000665
            2   1    lognormal  0.000369  0.000133
                2    lognormal  0.000423  0.000221
5      PID  1   1    lognormal  0.000902  0.000647
                2    lognormal  0.001313  0.000418
            2   1    lognormal  0.000456  0.000256
                2    lognormal  0.000621  0.000234

In [30]:
stripe = '10'
stripe_demands = raw_demands.loc[stripe,:]
temp_theta1 = list(stripe_demands.loc['PID', 'Theta_1'])
temp_theta0 = list(stripe_demands.loc['PID', 'Theta_0'])
temp_theta0

WARNING in T/ipykernel_24231/35214438.py at line 3
indexing past lexsort depth may impact performance.

WARNING in T/ipykernel_24231/35214438.py at line 4
indexing past lexsort depth may impact performance.



[0.005592655,
 0.009309350000000001,
 0.0016111300000000001,
 0.0030254599999999998]

In [31]:
for i in range (len(temp_theta1)):
    if temp_theta1[i] <= 0.0002:
        temp_theta1[i] = 0.0002
    if temp_theta0[i]<= 0.0004: 
        temp_theta0[i] = 0.0004
stripe_demands.loc['PID', 'Theta_1'] = temp_theta1
stripe_demands.loc['PID', 'Theta_0'] = temp_theta0

WARNING in T/ipykernel_24231/3573922843.py at line 6
indexing past lexsort depth may impact performance.

WARNING in T/ipykernel_24231/3573922843.py at line 7
indexing past lexsort depth may impact performance.



In [32]:
stripe_demands

Family   Theta_0   Theta_1
type loc dir                               
PID  1   1    lognormal  0.005593  0.009605
         2    lognormal  0.009309  0.021762
     2   1    lognormal  0.001611  0.001007
         2    lognormal  0.003025  0.001762
PFA  0   1    lognormal  0.215821  0.097217
         2    lognormal  0.215821  0.097217
     1   1    lognormal  1.007953  0.284006
         2    lognormal  1.040534  0.277568
     2   1    lognormal  1.116848  0.289781
         2    lognormal  1.166460  0.242678

In [33]:
stripe=1
PAL = Assessment({"PrintLog": True, "Seed": 42})
aa = pelicun_assessment1(PAL, baseDir, baselineID, raw_demands, stripe, delta_y=0.0075, sample_size=1000)

pelicun 3.1 | 

System Information:
         local time zone: PDT
         start time: 2023-10-10T13:10:48
         python: 3.8.2 (default, Mar 25 2020, 11:22:43) 
         [Clang 4.0.1 (tags/RELEASE_401/final)]
         numpy: 1.22.4
         pandas: 1.5.3
         
         --------------------------------------------------------------------------------
13:10:48 Assessement Started
WARNING in Codes/Loss_Pelicun/Run_pelicun_v3p1.py at line 66
indexing past lexsort depth may impact performance.

WARNING in Codes/Loss_Pelicun/Run_pelicun_v3p1.py at line 67
indexing past lexsort depth may impact performance.

WARNING in Codes/Loss_Pelicun/Run_pelicun_v3p1.py at line 71
indexing past lexsort depth may impact performance.

WARNING in Codes/Loss_Pelicun/Run_pelicun_v3p1.py at line 72
indexing past lexsort depth may impact performance.

WARNING in Codes/Loss_Pelicun/Run_pelicun_v3p1.py at line 81
indexing past lexsort depth may impact performance.

WARNING in Codes/Loss_Pelicun/Run_pelicun_v

In [35]:
aa.describe()

repair_cost  repair_time             
                       parallel   sequential
count  1000.000000  1000.000000  1000.000000
mean    109.368251     0.101499     0.102406
std     546.812998     0.560539     0.565101
min       0.000000     0.000000     0.000000
25%       0.000000     0.000000     0.000000
50%       0.000000     0.000000     0.000000
75%       0.000000     0.000000     0.000000
max    5857.739540     7.598384     7.598384

In [ ]:
ID = 's1_40x30_Moderate_HWS_GWB_Heavy'
baselineID = 's1_40x30'
pfa = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_N_20', ID, 'EDP_data', 'PFA.csv']), header=None)
sdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_N_20', ID, 'EDP_data', 'SDR.csv']), header=None)
rdr = pd.read_csv(os.path.join(baseDir, *['Results', 'Sampled_N_20', ID, 'EDP_data', 'RDR.csv']), header=None)
raw_demands = compile_demand_data([sdr, pfa], ['SDR', 'PFA'])
raw_demands.head()

In [ ]:
stripe=1
PAL = Assessment({"PrintLog": True, "Seed": 42})
aa = pelicun_assessment1(PAL, baseDir, baselineID, raw_demands, stripe, delta_y=0.0075, sample_size=1000)

In [ ]:
aa.head()

In [ ]:
pfa.shape

In [ ]:
index_name = []
median = []
log_std = []
#1-PFA-0-1
for stripe in pfa[0].unique():
    churn_df = pfa[pfa[0]==stripe]
    story_name = 0 
    for story in churn_df.columns:
        for uniq_dir in pfa[1].unique():
            if story >=3:
                story_name = story - 3
                index_name.append('%s-PFA-%s-%s'%(stripe, story_name, uniq_dir))
                median.append(pfa[(pfa[0]==stripe) & (pfa[1]==uniq_dir)][story].median())
                log_std.append(pfa[(pfa[0]==stripe) & (pfa[1]==uniq_dir)][story].std())
d = {
    'idx': index_name, 
    'median': median, 
    'log_std': log_std
}
df_pfa = pd.DataFrame(d)
df_pfa = df_pfa.set_index('idx')
df_pfa.head()

In [ ]:
df_pfa.shape

In [ ]:
index_name = []
median = []
log_std = []
#1-PFA-0-1
for stripe in sdr[0].unique():
    churn_df = sdr[sdr[0]==stripe]
    story_name = 0 
    for story in churn_df.columns:
        for uniq_dir in sdr[1].unique():
            if story >=3:
                story_name = story - 2
                index_name.append('%s-PID-%s-%s'%(stripe, story_name, uniq_dir))
                median.append(sdr[(sdr[0]==stripe) & (sdr[1]==uniq_dir)][story].median())
                log_std.append(sdr[(sdr[0]==stripe) & (sdr[1]==uniq_dir)][story].std())
d = {
    'idx': index_name, 
    'median': median, 
    'log_std': log_std
}
df_sdr = pd.DataFrame(d)
df_sdr = df_sdr.set_index('idx')
df_sdr.head()

In [ ]:
baseline_num = 3 if 'PFA'=='PFA' else 2
baseline_num

In [ ]:
df_sdr.shape

In [ ]:
df_pfa.shape

In [ ]:
df_comb = pd.concat([df_pfa, df_sdr])
df_comb.head()

In [ ]:
df_comb.shape

In [ ]:
raw_demands = convert_to_MultiIndex(df_comb, axis=0)
raw_demands.index.names = ['stripe','type','loc','dir']
raw_demands.head(n=30)

In [ ]:
os.getcwd()

In [ ]:
# raw_demands.to_csv('raw_demand_debug.csv')

In [67]:
stripe = '1'
stripe_demands = raw_demands.loc[stripe,:]
stripe_demands

Family   Theta_0   Theta_1
type loc dir                               
PID  1   1    lognormal  0.000248  0.000207
         2    lognormal  0.000332  0.000233
     2   1    lognormal  0.000164  0.000130
         2    lognormal  0.000224  0.000083
PFA  0   1    lognormal  0.066295  0.036617
         2    lognormal  0.066295  0.036617
     1   1    lognormal  0.114346  0.080800
         2    lognormal  0.104725  0.077485
     2   1    lognormal  0.174706  0.128308
         2    lognormal  0.157577  0.090025

In [37]:
# we'll use stripe 3 for this example
stripe = '10'
stripe_demands = raw_demands.loc[stripe,:]

# units - - - - - - - - - - - - - - - - - - - - - - - -  
stripe_demands.insert(0, 'Units',"")

# PFA is in "g" in this example, while PID is "rad"
stripe_demands.loc['PFA','Units'] = 'g'
stripe_demands.loc['PID','Units'] = 'rad'

# distribution family  - - - - - - - - - - - - - - - - -  
stripe_demands.insert(1, 'Family',"")

# we assume lognormal distribution for all demand marginals
stripe_demands['Family'] = 'lognormal'

# distribution parameters  - - - - - - - - - - - - - - -
# pelicun uses generic parameter names to handle various distributions within the same data structure
# we need to rename the parameter columns as follows:
# median -> theta_0
# log_std -> theta_1
stripe_demands.rename(columns = {'median': 'Theta_0'}, inplace=True)
stripe_demands.rename(columns = {'log_std': 'Theta_1'}, inplace=True)

stripe_demands

WARNING in T/ipykernel_24231/3558984128.py at line 9
indexing past lexsort depth may impact performance.

WARNING in T/ipykernel_24231/3558984128.py at line 10
indexing past lexsort depth may impact performance.



ValueError: cannot insert Family, already exists

In [68]:
# initialize a pelicun Assessment
PAL = Assessment({
    'Verbose':True,
    "PrintLog": True, 
    "Seed": 415})

pelicun 3.1 | 

System Information:
         local time zone: PDT
         start time: 2023-10-10T13:35:29
         python: 3.8.2 (default, Mar 25 2020, 11:22:43) 
         [Clang 4.0.1 (tags/RELEASE_401/final)]
         numpy: 1.22.4
         pandas: 1.5.3
         
         --------------------------------------------------------------------------------
13:35:29 Assessement Started


In [69]:
# prepare a correlation matrix that represents perfect correlation
ndims = stripe_demands.shape[0]
demand_types = stripe_demands.index 

perfect_CORR = pd.DataFrame(
    np.ones((ndims, ndims)),
    columns = demand_types,
    index = demand_types)

# load the demand model
PAL.demand.load_model({'marginals': stripe_demands,
                       'correlation': perfect_CORR})


         --------------------------------------------------------------------------------
13:35:30 Loading demand model...


AttributeError: 'NoneType' object has no attribute 'index'

In [ ]:
# perfect_CORR

In [41]:
# choose a sample size for the analysis
sample_size = 1000

# generate demand sample
PAL.demand.generate_sample({"SampleSize": sample_size})

# extract the generated sample
# Note that calling the save_sample() method is better than directly pulling the 
# sample attribute from the demand object because the save_sample method converts
# demand units back to the ones you specified when loading in the demands.
demand_sample = PAL.demand.save_sample()

demand_sample.head()

         --------------------------------------------------------------------------------
13:13:37 Generating sample from demand variables...
         
         10 random variables created.
         
         Correlations between 10 random variables successfully defined.
         
         Successfully generated 1000 realizations.
         --------------------------------------------------------------------------------


type       PFA                                                         PID                              
loc          0                   1                   2                   1                   2          
dir          1         2         1         2         1         2         1         2         1         2
0     0.230288  0.230288  1.218305  1.252294  1.355138  1.371535  0.005629  0.009446  0.001612  0.003029
1     0.221683  0.221683  1.090029  1.123268  1.209714  1.247154  0.005607  0.009365  0.001612  0.003027
2     0.203325  0.203325  0.846769  0.877600  0.934932  1.005094  0.005560  0.009186  0.001610  0.003022
3     0.226989  0.226989  1.168021  1.201754  1.298093  1.323017  0.005621  0.009415  0.001612  0.003028
4     0.228174  0.228174  1.185924  1.219754  1.318397  1.340325  0.005623  0.009426  0.001612  0.003029

In [42]:

# extract the generated sample
# Note that calling the save_sample() method is better than directly pulling the 
# sample attribute from the demand object because the save_sample method converts
# demand units back to the ones you specified when loading in the demands.
demand_sample = PAL.demand.save_sample()

demand_sample.head()

         --------------------------------------------------------------------------------


type       PFA                                                         PID                              
loc          0                   1                   2                   1                   2          
dir          1         2         1         2         1         2         1         2         1         2
0     0.230288  0.230288  1.218305  1.252294  1.355138  1.371535  0.005629  0.009446  0.001612  0.003029
1     0.221683  0.221683  1.090029  1.123268  1.209714  1.247154  0.005607  0.009365  0.001612  0.003027
2     0.203325  0.203325  0.846769  0.877600  0.934932  1.005094  0.005560  0.009186  0.001610  0.003022
3     0.226989  0.226989  1.168021  1.201754  1.298093  1.323017  0.005621  0.009415  0.001612  0.003028
4     0.228174  0.228174  1.185924  1.219754  1.318397  1.340325  0.005623  0.009426  0.001612  0.003029

In [ ]:
demand_sample.shape

In [ ]:

# datDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/woodSDA/Resources/PELICUN_example/Example01_FEMA_P58_Introduction'

# PAL.demand.save_sample(os.path.join(datDir, 'edp_demand_sample.csv'))

In [44]:
# get residual drift estimates 
delta_y = 0.0075
PID = demand_sample['PID']

RID = PAL.demand.estimate_RID(PID, {'yield_drift': delta_y})

# and join them with the demand_sample
demand_sample_ext = pd.concat([demand_sample, RID], axis=1)

# add spectral acceleration
# Sa_vals = [0.158, 0.387, 0.615, 0.843, 1.071, 1.299, 1.528, 1.756]
Sa_vals = [0.1, 0.2, 0.3, 0.4, 0.5, 0.7, 1.0, 1.2, 1.5, 1.8, 2.0, 2.5] #units: g
demand_sample_ext[('SA_1.13',0,1)] = Sa_vals[int(stripe)-1]

demand_sample_ext.describe().T

count      mean           std       min       25%       50%       75%       max
type    loc dir                                                                                  
PFA     0   1    1000.0  0.216843  2.112435e-02  0.157033  0.202160  0.215814  0.230411  0.296353
            2    1000.0  0.216843  2.112435e-02  0.157033  0.202160  0.215814  0.230411  0.296353
        1   1    1000.0  1.049378  3.038031e-01  0.398096  0.832674  1.007857  1.220205  2.545395
            2    1000.0  1.081345  3.056955e-01  0.419711  0.863320  1.040438  1.254203  2.573068
        2   1    1000.0  1.164673  3.443113e-01  0.432852  0.919056  1.116740  1.357294  2.874011
            2    1000.0  1.201275  2.956193e-01  0.527385  0.990781  1.166365  1.373363  2.574185
PID     1   1    1000.0  0.005593  5.370639e-05  0.005420  0.005557  0.005593  0.005629  0.005771
            2    1000.0  0.009312  2.026145e-04  0.008670  0.009174  0.009309  0.009447  0.009994
        2   1    1000.0  0.001611  1.621320e-06  0.001606  0.001610  0.001611  0.001612  0.001616
            2    1000.0  0.003025  5.329571e-06  0.003008  0.003022  0.003025  0.003029  0.003043
RID     1   1    1000.0  0.000000  0.000000e+00  0.000000  0.000000  0.000000  0.000000  0.000000
            2    1000.0  0.000554  1.286262e-04  0.000240  0.000464  0.000539  0.000631  0.001076
        2   1    1000.0  0.000000  0.000000e+00  0.000000  0.000000  0.000000  0.000000  0.000000
            2    1000.0  0.000000  0.000000e+00  0.000000  0.000000  0.000000  0.000000  0.000000
SA_1.13 0   1    1000.0  1.800000  4.443114e-16  1.800000  1.800000  1.800000  1.800000  1.800000

In [ ]:
# demand_sample_ext.to_csv('epd_demand_sample_with_residual.csv')

In [ ]:
# demand_sample_ext.head()

In [45]:
# add units to the data 
demand_sample_ext.T.insert(0, 'Units',"")

# PFA and SA are in "g" in this example, while PID and RID are "rad"
demand_sample_ext.loc['Units', ['PFA', 'SA_1.13']] = 'g'
demand_sample_ext.loc['Units',['PID', 'RID']] = 'rad'

display(demand_sample_ext)

PAL.demand.load_sample(demand_sample_ext)

type        PFA                                                         PID                                RID                     SA_1.13
loc           0                   1                   2                   1                   2              1              2            0
dir           1         2         1         2         1         2         1         2         1         2    1         2    1    2       1
0      0.230288  0.230288  1.218305  1.252294  1.355138  1.371535  0.005629  0.009446  0.001612  0.003029  0.0  0.000721  0.0  0.0     1.8
1      0.221683  0.221683  1.090029  1.123268  1.209714  1.247154  0.005607  0.009365  0.001612  0.003027  0.0  0.000451  0.0  0.0     1.8
2      0.203325  0.203325  0.846769    0.8776  0.934932  1.005094   0.00556  0.009186   0.00161  0.003022  0.0  0.000543  0.0  0.0     1.8
3      0.226989  0.226989  1.168021  1.201754  1.298093  1.323017  0.005621  0.009415  0.001612  0.003028  0.0  0.000537  0.0  0.0     1.8
4      0.228174  0.228174  1.185924  1.219754  1.318397  1.340325  0.005623  0.009426  0.001612  0.003029  0.0  0.000675  0.0  0.0     1.8
...         ...       ...       ...       ...       ...       ...       ...       ...       ...       ...  ...       ...  ...  ...     ...
996    0.191546  0.191546  0.711295  0.740113  0.782574  0.865983  0.005527  0.009064  0.001609  0.003019  0.0  0.000336  0.0  0.0     1.8
997    0.220212  0.220212  1.069034  1.102119  1.185946    1.2266  0.005604  0.009351  0.001611  0.003027  0.0  0.000378  0.0  0.0     1.8
998    0.239192  0.239192   1.36109  1.395552  1.517375  1.507766   0.00565  0.009526  0.001613  0.003031  0.0  0.000772  0.0  0.0     1.8
999    0.218567  0.218567   1.04587  1.078774  1.159732  1.203853    0.0056  0.009336  0.001611  0.003026  0.0  0.000581  0.0  0.0     1.8
Units         g         g         g         g         g         g       rad       rad       rad       rad  rad       rad  rad  rad       g

[1001 rows x 15 columns]

         --------------------------------------------------------------------------------
13:15:59 Loading demand data...
WARNING in Loss_Pelicun/pelicun_3_1/file_io.py at line 546
In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)

         Removing whitespace from header...
         Demand data successfully parsed.
         Removing whitespace from header...
         Demand units successfully parsed.


In [ ]:
# PAL.demand.save_sample(os.path.join(datDir, 'epd_demand_sample_with_residual.csv'))

## 3.3 Damage

### Define asset model

The asset model assigns components to the building and defines where they are and how much of each component is at each location. 

In [46]:
# load the component configuration
# cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)
# baselineID = 's4_96x48'
# baselineID = 's1_40x30'
cmp_marginals = pd.read_csv(os.path.join(baseDir, *['BuildingInfo',
                                                    baselineID,
                                                    'ComponentsList',
                                                    'components_list_marginals.csv']), index_col=1)

display(cmp_marginals.head(15))
print("...")
cmp_marginals.tail(10)

,cmpID,Description,Units,Normative Qty,Theta_0_normalized,Theta_0,Blocks,Family,Theta_1,Location,Direction,Incomplete,Changelog
ID,,,,,,,,,,,,,
B.10.71.002,B1071.002,Light framed wood walls with structural panel ...,SF,100,1.4400,144.0,2.0,NaN,NaN,1--2,"1,2",False,NaN
B.10.71.012,B1071.012,Light framed wood walls with structural panel ...,SF,100,1.4400,144.0,2.0,NaN,NaN,1--2,"1,2",False,NaN
B.20.11.121,B2011.121,Exterior Wall - Light framed wood walls with s...,LF,100,5.6000,560.0,2.0,NaN,NaN,1--2,"1,2",False,NaN
B.20.11.102,B2011.102,Exterior Wall - Light framed wood walls with s...,SF,100,5.6000,560.0,2.0,NaN,NaN,1--2,"1,2",False,NaN
B.30.11.012,B3011.012,"Clay roof tile, tiles secured and compliant wi...",SF,100,18.0000,1800.0,6.0,NaN,NaN,roof,1,False,NaN
C.10.11.011a,C1011.011a,"Wall Partition, Type: Gypsum with wood studs, ...",LF,100,1.8000,180.0,4.0,lognormal,0.4,1--2,"1,2",False,NaN
C.10.11.001a,C1011.001a,"Wall Partition, Type: Gypsum with metal studs,...",LF,100,1.8000,180.0,4.0,lognormal,0.4,1--2,"1,2",False,NaN
C.20.11.001b,C2011.001b,Prefabricated steel stair with steel treads an...,EA,2,1.0000,2.0,NaN,NaN,NaN,all,"1,2",False,NaN
D.20.21.013a,D2021.013a,Cold or Hot Potable - Small Diameter Threaded ...,LF,1000,0.1272,127.2,10.0,lognormal,0.4,all,"1,2",False,NaN


...


,cmpID,Description,Units,Normative Qty,Theta_0_normalized,Theta_0,Blocks,Family,Theta_1,Location,Direction,Incomplete,Changelog
ID,,,,,,,,,,,,,
D.30.67.012b,D3067.012b,Control Panel - Capacity: all - Equipment that...,EA,1,1.000,1.0,NaN,NaN,NaN,1,1,False,NaN
D.40.11.023a,D4011.023a,Fire Sprinkler Water Piping - Horizontal Mains...,LF,1000,1.272,1272.0,10.0,lognormal,0.4,all,1,False,NaN
D.40.11.074a,D4011.074a,Fire Sprinkler Drop Standard Threaded Steel - ...,EA,100,14.400,1440.0,10.0,lognormal,0.4,all,1,False,NaN
D.50.11.011a,D5011.011a,Transformer/primary service - Capacity: <100 k...,EA,1,1.000,1.0,NaN,NaN,NaN,1,1,False,NaN
D.50.12.013d,D5012.013d,Motor Control Center - Capacity: all - Equipme...,EA,1,1.000,1.0,NaN,NaN,NaN,1,1,False,NaN
D.50.12.023a,D5012.023a,Low Voltage Switchgear - Capacity: 100 to <350...,EA,1,1.000,1.0,NaN,NaN,NaN,1,1,True,"added 2.4g as MedianEDP, 0.4 as beta. Changed ..."
D.50.12.023b,D5012.023b,Low Voltage Switchgear - Capacity: 100 to <350...,EA,1,1.000,1.0,NaN,NaN,NaN,1,1,False,NaN
excessiveRID,NaN,excessive residual drift,EA,1,1.000,1.0,NaN,NaN,NaN,all,"1,2",NaN,NaN
collapse,NaN,collapsed building,EA,1,1.000,1.0,NaN,NaN,NaN,0,1,NaN,NaN


In [47]:
# to make the convenience keywords work in the model, 
# we need to specify the number of stories
PAL.stories = 2

# now load the model into Pelicun
PAL.asset.load_cmp_model({'marginals': cmp_marginals})

         --------------------------------------------------------------------------------
13:16:57 Loading component model...
         
         Parsing model file to characterize each component block
         Model parameters successfully parsed. 75 performance groups identified
         Converting model parameters to internal units...
         Model parameters successfully loaded.
         
         Component model marginal distributions:
                              Units Family    Theta_0  Theta_1  Blocks
         cmp          loc dir                                         
         B.10.71.002  1   1      SF    NaN  13.378038      NaN     2.0
                          2      SF    NaN  13.378038      NaN     2.0
                      2   1      SF    NaN  13.378038      NaN     2.0
                          2      SF    NaN  13.378038      NaN     2.0
         B.10.71.012  1   1      SF    NaN  13.378038      NaN     2.0
         ...                    ...    ...        ...     

In [ ]:
cmp_marginals.shape

In [ ]:
# let's take a look at the generated marginal parameters
# PAL.asset.cmp_marginal_params.loc['B.10.41.002a',:]
# PAL.asset.cmp_marginal_params.loc['B.10.71.001',:]

### 3.3.2 Sample asset distribution

In this example, the quantities are identical for every realization. We still need to generate a component quantity sample because the calculations in Pelicun expect an array of component quantity realizations. The sample size for the component quantities is automatically inferred from the demand sample. If such a sample is not available, you need to provide a sample size as the first argument of the generate_cmp_sample method.

The table below shows the statistics for each Performance Group's quantities. Notice the zero standard deviation and that the minimum and maximum values are identical - this confirms that the quantities are deterministic.

We could edit this sample and load the edited version back to Pelicun like we did for the Demands earlier.

In [48]:
# Generate the component quantity sample
PAL.asset.generate_cmp_sample()

# get the component quantity sample - again, use the save function to convert units
cmp_sample = PAL.asset.save_cmp_sample()

cmp_sample.describe()

         --------------------------------------------------------------------------------
13:17:05 Generating sample from component quantity variables...
         
         75 random variables created.
         
         Successfully generated 1000 realizations.
         --------------------------------------------------------------------------------


cmp   B.10.71.002                         B.10.71.012                         B.20.11.102                         B.20.11.121                         B.30.11.012 C.10.11.001a                                        C.10.11.011a                                        C.20.11.001b                  \
loc             1               2                   1               2                   1               2                   1               2                   3            1                         2                         1                         2                         1               2   
dir             1       2       1       2           1       2       1       2           1       2       1       2           1       2       1       2           1            1            2            1            2            1            2            1            2            1       2       1   
count      1000.0  1000.0  1000.0  1000.0      1000.0  1000.0  1000.0  1000.0      1000.0  1000.0  1000.0  1000.0      1000.0  1000.0  1000.0  1000.0      1000.0  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000  1000.000000       1000.0  1000.0  1000.0   
mean        144.0   144.0   144.0   144.0       144.0   144.0   144.0   144.0       560.0   560.0   560.0   560.0       560.0   560.0   560.0   560.0      1800.0   195.010161   195.060708   194.974354   194.959075   194.914828   194.984183   194.974177   194.964425          2.0     2.0     2.0   
std           0.0     0.0     0.0     0.0         0.0     0.0     0.0     0.0         0.0     0.0     0.0     0.0         0.0     0.0     0.0     0.0         0.0    81.468600    81.680184    81.143454    81.021466    80.803543    81.081172    81.080795    81.008622          0.0     0.0     0.0   
min         144.0   144.0   144.0   144.0       144.0   144.0   144.0   144.0       560.0   560.0   560.0   560.0       560.0   560.0   560.0   560.0      1800.0    30.003812    51.588074    49.750309    46.147746    46.494356    49.653344    48.876533    51.759538          2.0     2.0     2.0   
25%         144.0   144.0   144.0   144.0       144.0   144.0   144.0   144.0       560.0   560.0   560.0   560.0       560.0   560.0   560.0   560.0      1800.0   137.411390   137.559482   137.415493   137.450519   137.522247   137.455783   137.408103   137.502413          2.0     2.0     2.0   
50%         144.0   144.0   144.0   144.0       144.0   144.0   144.0   144.0       560.0   560.0   560.0   560.0       560.0   560.0   560.0   560.0      1800.0   179.991340   179.982798   179.932509   180.008175   180.028187   180.028931   179.967657   180.043306          2.0     2.0     2.0   
75%         144.0   144.0   144.0   144.0       144.0   144.0   144.0   144.0       560.0   560.0   560.0   560.0       560.0   560.0   560.0   560.0      1800.0   235.755151   235.589739   235.623069   235.669161   235.622567   235.653519   235.760848   235.597453          2.0     2.0     2.0   
max         144.0   144.0   144.0   144.0       144.0   144.0   144.0   144.0       560.0   560.0   560.0   560.0       560.0   560.0   560.0   560.0      1800.0   735.449968   777.343848   692.027032   646.348237   638.339743   634.398746   667.200405   643.826988          2.0     2.0     2.0   

cmp           D.20.21.013a                                        D.20.21.013b                                        D.20.21.023a                                        D.20.21.023b                                        D.20.31.013b                                        D.30.31.013c  \
loc                      1                         2                         1                         2                         1                         2                         1                         2                         1                         2                         3   
dir         2            1            2            1            2            1            2            1            2            1           

In [49]:
cmp_sample.shape

(1000, 75)

In [ ]:
# PAL.asset.save_cmp_sample(os.path.join(datDir, 'component_sample.csv'))

### 3.3.3 Define component fragilities

In [50]:
# review the damage model - in this example: fragility functions
P58_data = PAL.get_default_data('fragility_DB_FEMA_P58_2nd')

display(P58_data.head(3))

print(P58_data['Incomplete'].sum(),' incomplete component fragility definitions')

Demand                                          Incomplete                  LS1                                           LS2                                           LS3                                           LS4                       
             Directional Offset                         Type Unit              DamageStateWeights     Family Theta_0 Theta_1 DamageStateWeights     Family Theta_0 Theta_1 DamageStateWeights     Family Theta_0 Theta_1 DamageStateWeights Family Theta_0 Theta_1
ID                                                                                                                                                                                                                                                                
B.10.31.001          1.0    0.0  Peak Interstory Drift Ratio   ea        0.0  0.950000 | 0.050000  lognormal    0.04     0.4                NaN  lognormal    0.08     0.4                NaN  lognormal    0.11     0.4                NaN    NaN     NaN     NaN
B.10.31.011a         1.0    0.0  Peak Interstory Drift Ratio   ea        0.0  0.950000 | 0.050000  lognormal    0.04     0.4                NaN  lognormal    0.07     0.4                NaN  lognormal    0.10     0.4                NaN    NaN     NaN     NaN
B.10.31.011b         1.0    0.0  Peak Interstory Drift Ratio   ea        0.0  0.950000 | 0.050000  lognormal    0.04     0.4                NaN  lognormal    0.07     0.4                NaN  lognormal    0.10     0.4                NaN    NaN     NaN     NaN

179.0  incomplete component fragility definitions


In [51]:
# note that we drop the last three components here (excessiveRID, irreparable, and collapse) 
# because they are not part of P58
cmp_list = cmp_marginals.index.unique().values[:-3]

P58_data_for_this_assessment = P58_data.loc[cmp_list,:].sort_values('Incomplete', ascending=False)

additional_fragility_db = P58_data_for_this_assessment.loc[
    P58_data_for_this_assessment['Incomplete'] == 1].sort_index() 

additional_fragility_db

Empty DataFrame
Columns: [(Demand, Directional), (Demand, Offset), (Demand, Type), (Demand, Unit), (Incomplete, ), (LS1, DamageStateWeights), (LS1, Family), (LS1, Theta_0), (LS1, Theta_1), (LS2, DamageStateWeights), (LS2, Family), (LS2, Theta_0), (LS2, Theta_1), (LS3, DamageStateWeights), (LS3, Family), (LS3, Theta_0), (LS3, Theta_1), (LS4, DamageStateWeights), (LS4, Family), (LS4, Theta_0), (LS4, Theta_1)]
Index: []

In [ ]:
additional_fragility_db.index

In [ ]:
# P58_metadata = PAL.get_default_metadata('fragility_DB_FEMA_P58_2nd')

# pprint.pprint(P58_metadata['D.20.22.013b'])

In [ ]:
# additional_fragility_db.loc[['D.20.31.013b', 'D.30.31.013a', 'D.30.41.103a',
#                              'D.30.52.013a', 'D.30.67.012a', 'D.50.12.013d',
#                              'D.50.12.023a'], 
#                            [('LS1','Theta_1'),('LS2','Theta_1')]] = 0.4
# additional_fragility_db.loc[['D.20.31.013b', 'D.30.31.013a', 'D.30.41.103a',
#                              'D.30.52.013a', 'D.30.67.012a', 'D.50.12.013d',
#                              'D.50.12.023a'], 
#                            [('LS1','Theta_0'),('LS2','Theta_0')]] = 1.5 #g

In [ ]:
# # We can set the incomplete flag to 0 for these components
# additional_fragility_db['Incomplete'] = 0
# additional_fragility_db

In [ ]:
# # D2022.013a, 023a, 023b - Heating, hot water piping and bracing
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc[['D.20.22.013a','D.20.22.023a','D.20.22.023b'],
#                             [('LS1','Theta_1'),('LS2','Theta_1')]] = 0.5

# # D2031.013b - Sanitary Waste piping
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc['D.20.31.013b',('LS1','Theta_1')] = 0.5

# # D2061.013b - Steam piping
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc['D.20.61.013b',('LS1','Theta_1')] = 0.5

# # D3031.013i - Chiller
# # use a placeholder of 1.5|0.5
# additional_fragility_db.loc['D.30.31.013i',('LS1','Theta_0')] = 1.5 #g
# additional_fragility_db.loc['D.30.31.013i',('LS1','Theta_1')] = 0.5

# # D3031.023i - Cooling Tower
# # use a placeholder of 1.5|0.5
# additional_fragility_db.loc['D.30.31.023i',('LS1','Theta_0')] = 1.5 #g
# additional_fragility_db.loc['D.30.31.023i',('LS1','Theta_1')] = 0.5

# # D3052.013i - Air Handling Unit
# # use a placeholder of 1.5|0.5
# additional_fragility_db.loc['D.30.52.013i',('LS1','Theta_0')] = 1.5 #g
# additional_fragility_db.loc['D.30.52.013i',('LS1','Theta_1')] = 0.5

# # We can set the incomplete flag to 0 for these components
# additional_fragility_db['Incomplete'] = 0

# additional_fragility_db

In [52]:

# irreparable damage
# this is based on the default values in P58
additional_fragility_db.loc[
    'excessiveRID', [('Demand','Directional'),
                    ('Demand','Offset'),
                    ('Demand','Type'), 
                    ('Demand','Unit')]] = [1, 0, 'Residual Interstory Drift Ratio', 'rad']   

additional_fragility_db.loc[
    'excessiveRID', [('LS1','Family'),
                    ('LS1','Theta_0'),
                    ('LS1','Theta_1')]] = ['lognormal', 0.01, 0.3]   

additional_fragility_db.loc[
    'irreparable', [('Demand','Directional'),
                    ('Demand','Offset'),
                    ('Demand','Type'), 
                    ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   


# a very high capacity is assigned to avoid damage from demands
additional_fragility_db.loc[
    'irreparable', ('LS1','Theta_0')] = 1e10 

# collapse
# capacity is assigned based on the example in the FEMA P58 background documentation
additional_fragility_db.loc[
    'collapse', [('Demand','Directional'),
                 ('Demand','Offset'),
                 ('Demand','Type'), 
                 ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   


additional_fragility_db.loc[
    'collapse', [('LS1','Family'),
                 ('LS1','Theta_0'),
                 ('LS1','Theta_1')]] = ['lognormal', 1.35, 0.5]  

# We set the incomplete flag to 0 for the additional components
additional_fragility_db['Incomplete'] = 0

additional_fragility_db.tail(3)

Demand                                              Incomplete                LS1                                                 LS2                                       LS3                                       LS4                       
             Directional Offset                             Type Unit            DamageStateWeights     Family       Theta_0 Theta_1 DamageStateWeights Family Theta_0 Theta_1 DamageStateWeights Family Theta_0 Theta_1 DamageStateWeights Family Theta_0 Theta_1
ID                                                                                                                                                                                                                                                                
excessiveRID         1.0    0.0  Residual Interstory Drift Ratio  rad          0                NaN  lognormal  1.000000e-02     0.3                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN
irreparable          1.0    0.0  Peak Spectral Acceleration|1.13    g          0                NaN        NaN  1.000000e+10     NaN                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN
collapse             1.0    0.0  Peak Spectral Acceleration|1.13    g          0                NaN  lognormal  1.350000e+00     0.5                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN                NaN    NaN     NaN     NaN

### 3.3.4 Load component fragility data

In [53]:
PAL.damage.load_damage_model([
    additional_fragility_db,  # This is the extra fragility data we've just created
    'PelicunDefault/fragility_DB_FEMA_P58_2nd.csv' # and this is a table with the default P58 data    
])

         --------------------------------------------------------------------------------
13:18:25 Loading damage model...
         Damage model parameters successfully parsed.


In [54]:
# PAL.damage.load_damage_model([
# #     additional_fragility_db,  # This is the extra fragility data we've just created
#     'PelicunDefault/fragility_DB_FEMA_P58_2nd.csv' # and this is a table with the default P58 data    
# ])

## 3.3.5 Damage Process

In [55]:
# FEMA P58 uses the following process:
dmg_process = {
    "1_collapse": {
        "DS1": "ALL_NA"
    },
    "2_excessiveRID": {
        "DS1": "irreparable_DS1"
    }
}

In [56]:
# Now we can run the calculation
PAL.damage.calculate(dmg_process=dmg_process)#, block_batch_size=100) #- for large calculations

         --------------------------------------------------------------------------------
13:18:40 Calculating damages...
         Number of Performance Groups in Asset Model: 75
         Number of Component Blocks: 264
         1 batches of Performance Groups prepared for damage assessment
13:18:40 Calculating damage for PG batch 1 with 264 blocks
13:18:40 Generating capacity variables ...
         2x518 random variables created.
13:18:40 Sampling capacities...
13:18:40 Raw samples are available
13:18:40 Successfully generated 1000 realizations.
13:18:40 Collecting required demand information...
13:18:40 Assembling demand data for calculation...
13:18:40 Evaluating damage states...
13:18:41 Calculating damage quantities...
         Raw damage calculation successful.
13:18:41 Applying damage processes...
13:18:41 Applying task...
         Damage process task successfully applied.
13:18:41 Applying task...
         Damage process task successfully applied.
         Damage processes succ

In [ ]:
# PAL.damage.save_sample(os.path.join(datDir, 'Damage_sample.csv'))

In [57]:
damage_sample = PAL.damage.save_sample()

print("Size of damage results: ", sys.getsizeof(damage_sample)/1024/1024, "MB")

         --------------------------------------------------------------------------------
13:18:55 Saving damage sample...
Size of damage results:  1.4076347351074219 MB


In [58]:
damage_sample.shape

(1000, 180)

#### Damage summary statistics of a component type

In [ ]:
# component = 'B.20.22.031'
# damage_sample.describe([0.1, 0.5, 0.9]).T.loc[component,:].head(30)

In [59]:
damage_sample.head()

cmp B.10.71.002                                B.10.71.012                                                           B.20.11.102                                                             B.20.11.121                                B.30.11.012              C.10.11.001a                             \
loc           1                       2                  1                                   2                                 1                                     2                                 1                       2                  3                         1                              
dir           1           2           1      2           1                2                  1         2                       1                  2                  1         2                       1           2           1      2           1                         1                     2        
ds            0    1      0    1      0      0           0     1     2    0    1      2      0    1    0      1    2           0      1      2    0    1      2      0    1    0      1    2           0    1      0    1      0      0           0    1       2            0           1    2    0    1   
0           NaN  NaN    NaN  NaN    NaN    NaN         NaN   NaN   NaN  NaN  NaN    NaN    NaN  NaN  NaN    NaN  NaN         NaN    NaN    NaN  NaN  NaN    NaN    NaN  NaN  NaN    NaN  NaN         NaN  NaN    NaN  NaN    NaN    NaN         NaN  NaN     NaN          NaN         NaN  NaN  NaN  NaN   
1           NaN  NaN    NaN  NaN    NaN    NaN         NaN   NaN   NaN  NaN  NaN    NaN    NaN  NaN  NaN    NaN  NaN         NaN    NaN    NaN  NaN  NaN    NaN    NaN  NaN  NaN    NaN  NaN         NaN  NaN    NaN  NaN    NaN    NaN         NaN  NaN     NaN          NaN         NaN  NaN  NaN  NaN   
2           NaN  NaN    NaN  NaN    NaN    NaN         NaN   NaN   NaN  NaN  NaN    NaN    NaN  NaN  NaN    NaN  NaN         NaN    NaN    NaN  NaN  NaN    NaN    NaN  NaN  NaN    NaN  NaN         NaN  NaN    NaN  NaN    NaN    NaN         NaN  NaN     NaN          NaN         NaN  NaN  NaN  NaN   
3         144.0  0.0  144.0  0.0  144.0  144.0         0.0  72.0  72.0  0.0  0.0  144.0  144.0  0.0  0.0  144.0  0.0         0.0  280.0  280.0  0.0  0.0  560.0  560.0  0.0  0.0  560.0  0.0       560.0  0.0  560.0  0.0  560.0  560.0       600.0  0.0  1200.0    91.635084  274.905252  0.0  0.0  0.0   
4           NaN  NaN    NaN  NaN    NaN    NaN         NaN   NaN   NaN  NaN  NaN    NaN    NaN  NaN  NaN    NaN  NaN         NaN    NaN    NaN  NaN  NaN    NaN    NaN  NaN  NaN    NaN  NaN         NaN  NaN    NaN  NaN    NaN    NaN         NaN  NaN     NaN          NaN         NaN  NaN  NaN  NaN   

cmp                                                          C.10.11.011a                                                                                                                      C.20.11.001b                                                        D.20.21.013a                            \
loc                            2                                        1                                                                        2                                                        1                                  2                                1                             
dir                            1                2                       1                                2                                       1                     2                                  1              2                   1         2                      1                         2   
ds            2    3           0    1           0          1            0           1          2    3    0    1           2          3           0    1    2           0           1    2    3            0    1    2    0    1    2    3    0    1    0    1    2            0        1    2           0   
0           NaN  NaN         NaN  NaN         NaN        NaN          NaN         NaN        Na

In [ ]:
# damage_sample.to_csv('damage_sample.csv')

In [ ]:
damage_sample.columns

In [ ]:
# component = 'D.10.14.011'
# # damage_sample.describe([0.1, 0.5, 0.9]).T.loc[component,:].head(30)
# dmg_plot = damage_sample.loc[:, component].groupby(level=[0,2], axis=1).sum().T

# px.bar(x=dmg_plot.index.get_level_values(1), y=dmg_plot.mean(axis=1), 
#        color=dmg_plot.index.get_level_values(0),
#        barmode='group',
#        labels={
#            'x':'Damage State',
#            'y':'Component Quantity [ft2]',
#            'color': 'Floor'
#        },
#        title=f'Mean Quantities of component {component} in each Damage State',
#        height=500
#       )

In [60]:
print('Probability of collapse: ', damage_sample[('collapse','0','1','0')].mean())
print()
print('Probability of irreparable damage: ', damage_sample[('irreparable','0','1','0')].mean())

Probability of collapse:  0.282

Probability of irreparable damage:  1.0


### Consequence mapping to damages

In [61]:
# let us prepare the map based on the component list

# we need to prepend 'DMG-' to the component names to tell pelicun to look for the damage of these components
drivers = [f'DMG-{cmp}' for cmp in cmp_marginals.index.unique()]
drivers = drivers[:-3]+drivers[-2:]

# we are looking at repair consequences in this example
# the components in P58 have consequence models under the same name
loss_models = cmp_marginals.index.unique().tolist()[:-3]

# We will define the replacement consequence in the following cell.
loss_models+=['replacement',]*2

# Assemble the DataFrame with the mapping information
# The column name identifies the type of the consequence model.
loss_map = pd.DataFrame(loss_models, columns=['BldgRepair'], index=drivers)

loss_map

,BldgRepair
DMG-B.10.71.002,B.10.71.002
DMG-B.10.71.012,B.10.71.012
DMG-B.20.11.121,B.20.11.121
DMG-B.20.11.102,B.20.11.102
DMG-B.30.11.012,B.30.11.012
...,...
DMG-D.50.12.013d,D.50.12.013d
DMG-D.50.12.023a,D.50.12.023a
DMG-D.50.12.023b,D.50.12.023b
DMG-collapse,replacement


In [62]:
# load the consequence models
P58_data = PAL.get_default_data('bldg_repair_DB_FEMA_P58_2nd')

# get the consequences used by this assessment
P58_data_for_this_assessment = P58_data.loc[loss_map['BldgRepair'].values[:-2],:]

print(P58_data_for_this_assessment['Incomplete'].sum(), ' components have incomplete consequence models assigned.')

display(P58_data_for_this_assessment.head(30))

0.0  components have incomplete consequence models assigned.


DS1                                                  DS10                                DS11                                DS12                                DS13                                DS14                                DS15                               \
                      Family LongLeadTime                 Theta_0   Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1 Family LongLeadTime Theta_0 Theta_1   
B.10.71.002  Cost  lognormal          NaN     2134.17,1313.33|3,8  0.188729    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
             Time  lognormal          0.0     2.06801,1.27132|3,8  0.313239    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
B.10.71.012  Cost  lognormal          NaN           2990,1840|2,6  0.256634    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
             Time  lognormal          0.0     2.89971,1.78353|2,6  0.358275    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
B.20.11.121  Cost  lognormal          NaN     2134.17,1313.33|3,8  0.188729    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
...                      ...          ...                     ...       ...    ...          ...     ...     ...    ...          ...     ...     ...    ...          ...     ...     ...    ...          ...     ...     ...    ...          ...     ...     ...    ...          ...     ...     ...   
D.20.31.013b Time  lognormal          0.0  0.847029,0.254088|5,10  0.627716    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
D.30.31.013c Cost  lognormal          NaN            1100,900|1,5  0.452310    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
             Time  lognormal          0.0     1.29415,1.05885|1,5  0.516802    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
D.30.41.011c Cost  lognormal          NaN             715,585|1,5  0.367364    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   
             Time  lognormal          0.0   0.841171,0.688229|1,5  0.444361    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN    NaN          NaN     NaN     NaN   

                         DS2                                                    DS3                                                DS4                                 DS5                   

### adding custom consequence functions

we need to define the replacement consequence for the collapse and irreparable damage cases.

In [63]:
# initialize the dataframe
additional_consequences = pd.DataFrame(
    columns = pd.MultiIndex.from_tuples([('Incomplete',''), 
                                         ('Quantity','Unit'), 
                                         ('DV', 'Unit'), 
                                         ('DS1', 'Theta_0')]),
    index=pd.MultiIndex.from_tuples([('replacement','Cost'), 
                                     ('replacement','Time')])
)

# add the data about replacement cost and time
additional_consequences.loc[('replacement', 'Cost')] = [0, '1 EA', 'USD_2011', 21600000]
additional_consequences.loc[('replacement', 'Time')] = [0, '1 EA', 'worker_day', 12500]  

additional_consequences

Incomplete Quantity          DV       DS1
                                Unit        Unit   Theta_0
replacement Cost          0     1 EA    USD_2011  21600000
            Time          0     1 EA  worker_day     12500

### 3.4.3 Load component consequence data

In [64]:
# Load the loss model to pelicun
PAL.bldg_repair.load_model(
    [additional_consequences,
     "PelicunDefault/bldg_repair_DB_FEMA_P58_2nd.csv"], 
    loss_map)

         --------------------------------------------------------------------------------
13:20:08 Loading loss map for BldgRepair...
         Loss map successfully parsed.
         --------------------------------------------------------------------------------
13:20:08 Loading loss parameters for BldgRepair...
         Loss parameters successfully parsed.


### 3.4.4 Loss calculation

Loss calculation in Pelicun requires 
- a pre-assigned set of component consequence functions;
- a pre-assigned sample of demands and/or damages;
- and a loss mapping matrix

The sample size for the loss calculation is automatically inferred from the demand/damage sample size.

In [65]:
PAL.bldg_repair.calculate()

         --------------------------------------------------------------------------------
13:20:13 Calculating losses...
         
         Aggregating damage quantities...
         Successfully aggregated damage quantities.
         
         Calculating the median repair consequences...
         Successfully determined median repair consequences.
13:20:13 
         Considering deviations from the median values to obtain random DV sample...
         Preparing random variables for repair cost and time...
         
         208 random variables created.
         
         Successfully generated 1000 realizations of deviation from the median consequences.
         Successfully obtained DV sample.
13:20:14 Loss calculation successful.


In [ ]:
# PAL.bldg_repair.save_sample(os.path.join(datDir, 'loss_sample.csv'))

In [66]:
loss_sample = PAL.bldg_repair.sample
loss_sample_cost = loss_sample['COST']
loss_sample_time = loss_sample['TIME']
loss_sample_cost.head()

loss B.10.71.002       B.10.71.012                                                       B.20.11.102                                                      B.20.11.121      B.30.11.012               C.10.11.001a                                            C.10.11.011a                          \
dmg  B.10.71.002       B.10.71.012                                                       B.20.11.102                                                      B.20.11.121      B.30.11.012               C.10.11.001a                                            C.10.11.011a                           
ds             1                 1                                  2                              1                                  2                             1                1             2            1                         2                3            1                           
loc            1                 1         2                        1                 2            1         2                        1                 2           1                3             3            1         2               1                1            1         2                 
dir            1    2            1    2    1            2           1            2    2            1    2    1            2           1            2    2           1    2           1             1            1    2    1          2    1             2  2            1    2    1             2   
0            0.0  0.0          0.0  0.0  0.0          0.0         0.0          0.0  0.0          0.0  0.0  0.0          0.0         0.0          0.0  0.0         0.0  0.0         0.0           0.0          0.0  0.0  0.0        0.0  0.0           0.0  0          0.0  0.0  0.0           0.0   
1            0.0  0.0          0.0  0.0  0.0          0.0         0.0          0.0  0.0          0.0  0.0  0.0          0.0         0.0          0.0  0.0         0.0  0.0         0.0           0.0          0.0  0.0  0.0        0.0  0.0           0.0  0          0.0  0.0  0.0           0.0   
2            0.0  0.0          0.0  0.0  0.0          0.0         0.0          0.0  0.0          0.0  0.0  0.0          0.0         0.0          0.0  0.0         0.0  0.0         0.0           0.0          0.0  0.0  0.0        0.0  0.0           0.0  0          0.0  0.0  0.0           0.0   
3            0.0  0.0  2053.294557  0.0  0.0  4396.175706  2207.08583  7216.564323  0.0  4759.286241  0.0  0.0  6837.806228  5309.84632  9887.249716  0.0         0.0  0.0         0.0  44259.145081  7998.998233  0.0  0.0  839.05032  0.0  15858.094331  0  9908.727329  0.0  0.0  14342.659346   
4            0.0  0.0          0.0  0.0  0.0          0.0         0.0          0.0  0.0          0.0  0.0  0.0          0.0         0.0          0.0  0.0         0.0  0.0         0.0           0.0          0.0  0.0  0.0        0.0  0.0           0.0  0          0.0  0.0  0.0           0.0   

loss                                                                C.20.11.001b                                           D.20.21.013a                                                            D.20.21.013b                          D.20.21.023a                                                     \
dmg                                                                 C.20.11.001b                                           D.20.21.013a                                                            D.20.21.013b                          D.20.21.023a                                                      
ds               2                            3                                1                            2            3            1                                     2                                 1                                     1                                             2        
loc              1                  2         1                  2             1                  2         1       2    1            1                     2            

In [ ]:
# loss_sample_cost.to_csv('loss_sample_cost.csv')
# loss_sample_time.to_csv('loss_sample_time.csv')

**Aggregate losses**

Aggregating losses for repair costs is straightforward, but repair times are less trivial. Pelicun adopts the method from FEMA P-58 and provides two bounding values for aggregate repair times:
- **parallel** assumes that repairs are conducted in parallel across locations. In each location, repairs are assumed to be sequential. This translates to aggregating component repair times by location and choosing the longest resulting aggregate value across locations.
- **sequential** assumes repairs are performed sequentially across locations and within each location. This translates to aggregating component repair times across the entire building.

The parallel option is considered a lower bound and the sequential is an upper bound of the real repair time. Pelicun automatically calculates both options for all (i.e., not only FEMA P-58) analyses.

In [ ]:
agg_DF = PAL.bldg_repair.aggregate_losses()
agg_DF.head()

In [ ]:
# agg_DF.to_csv('aggregate_loss.csv')

In [ ]:
agg_DF.describe()

# 4 Concise version

All scripts required to run this example are collected to a few cells below for your convenience.

In [ ]:
#parameters
sample_size = 10000
delta_y = 0.0075
stripe = '1'
PAL = Assessment({"PrintLog": True, "Seed": 415,})

## 4.1 Pre-processing

In [ ]:
# prepare demand input
raw_demands = convert_to_MultiIndex(df_comb, axis=0)
raw_demands.index.names = ['stripe','type','loc','dir']

# prepare the demand input for pelicun
stripe_demands = raw_demands.loc[stripe,:]

# units - - - - - - - - - - - - - - - - - - - - - - - -  
stripe_demands.insert(0, 'Units',"")
stripe_demands.loc['PFA','Units'] = 'g'
stripe_demands.loc['PID','Units'] = 'rad'

# distribution family  - - - - - - - - - - - - - - - - -  
stripe_demands.insert(1, 'Family',"")
stripe_demands['Family'] = 'lognormal'

# distribution parameters  - - - - - - - - - - - - - - -
stripe_demands.rename(columns = {'median': 'Theta_0'}, inplace=True)
stripe_demands.rename(columns = {'log_std': 'Theta_1'}, inplace=True)


# prepare a correlation matrix that represents perfect correlation
ndims = stripe_demands.shape[0]
demand_types = stripe_demands.index 

perfect_CORR = pd.DataFrame(
    np.ones((ndims, ndims)),
    columns = demand_types,
    index = demand_types)

# prepare additional fragility and consequence data ahead of time
# cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)
# baselineID = 's4_96x48'
cmp_marginals = pd.read_csv(os.path.join(baseDir, *['BuildingInfo',
                                                    baselineID,
                                                    'ComponentsList',
                                                    'components_list_marginals.csv']), index_col=1)

# add missing data to P58 damage model
P58_data = PAL.get_default_data('fragility_DB_FEMA_P58_2nd')
cmp_list = cmp_marginals.index.unique().values[:-3]

# now take those components that are incomplete, and add the missing information
additional_fragility_db = P58_data.loc[cmp_list,:].loc[P58_data.loc[cmp_list,'Incomplete'] == 1].sort_index()

# # D2022.013a, 023a, 023b - Heating, hot water piping and bracing
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc[['D.20.22.013a','D.20.22.023a','D.20.22.023b'],
#                             [('LS1','Theta_1'),('LS2','Theta_1')]] = 0.5

# # D2031.013b - Sanitary Waste piping
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc['D.20.31.013b',('LS1','Theta_1')] = 0.5

# # D2061.013b - Steam piping
# # dispersion values are missing, we use 0.5
# additional_fragility_db.loc['D.20.61.013b',('LS1','Theta_1')] = 0.5

# # D3031.013i - Chiller
# # use a placeholder of 3.0|0.5
# additional_fragility_db.loc['D.30.31.013i',('LS1','Theta_0')] = 3.0
# additional_fragility_db.loc['D.30.31.013i',('LS1','Theta_1')] = 0.5

# # D3031.023i - Cooling Tower
# # use a placeholder of 3.0|0.5
# additional_fragility_db.loc['D.30.31.023i',('LS1','Theta_0')] = 3.0
# additional_fragility_db.loc['D.30.31.023i',('LS1','Theta_1')] = 0.5

# # D3052.013i - Air Handling Unit
# # use a placeholder of 3.0|0.5
# additional_fragility_db.loc['D.30.52.013i',('LS1','Theta_0')] = 3.0
# additional_fragility_db.loc['D.30.52.013i',('LS1','Theta_1')] = 0.5

# prepare the extra damage models for collapse and irreparable damage
additional_fragility_db.loc[
    'excessiveRID', [('Demand','Directional'),
                    ('Demand','Offset'),
                    ('Demand','Type'), 
                    ('Demand','Unit')]] = [1, 0, 'Residual Interstory Drift Ratio', 'rad']   

additional_fragility_db.loc[
    'excessiveRID', [('LS1','Family'),
                    ('LS1','Theta_0'),
                    ('LS1','Theta_1')]] = ['lognormal', 0.01, 0.3]   

additional_fragility_db.loc[
    'irreparable', [('Demand','Directional'),
                    ('Demand','Offset'),
                    ('Demand','Type'), 
                    ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   

additional_fragility_db.loc[
    'irreparable', ('LS1','Theta_0')] = 1e10

additional_fragility_db.loc[
    'collapse', [('Demand','Directional'),
                 ('Demand','Offset'),
                 ('Demand','Type'), 
                 ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   

additional_fragility_db.loc[
    'collapse', [('LS1','Family'),
                 ('LS1','Theta_0'),
                 ('LS1','Theta_1')]] = ['lognormal', 1.35, 0.5]  

# Now we can set the incomplete flag to 0 for these components
additional_fragility_db['Incomplete'] = 0

# create the additional consequence models
additional_consequences = pd.DataFrame(
    columns = pd.MultiIndex.from_tuples([
        ('Incomplete',''), ('Quantity','Unit'), ('DV', 'Unit'), ('DS1', 'Theta_0')]),
    index=pd.MultiIndex.from_tuples([
        ('replacement','Cost'), ('replacement','Time')])
)

additional_consequences.loc[('replacement', 'Cost')] = [0, '1 EA', 'USD_2011', 21600000]
additional_consequences.loc[('replacement', 'Time')] = [0, '1 EA', 'worker_day', 12500]

## 4.2 Demands

In [ ]:
# initialize a pelicun Assessment
# PAL = Assessment({"PrintLog": True, "Seed": 415,})

# load the demand model
PAL.demand.load_model({'marginals': stripe_demands,
                       'correlation': perfect_CORR})

# generate samples
PAL.demand.generate_sample({"SampleSize": sample_size})

# add residual drift and Sa
demand_sample = PAL.demand.save_sample()

RID = PAL.demand.estimate_RID(demand_sample['PID'], {'yield_drift': delta_y})
demand_sample_ext = pd.concat([demand_sample, RID], axis=1)

Sa_vals = [0.158, 0.387, 0.615, 0.843, 1.071, 1.299, 1.528, 1.756]
demand_sample_ext[('SA_1.13',0,1)] = Sa_vals[int(stripe)-1]

# add units to the data 
demand_sample_ext.T.insert(0, 'Units',"")

# PFA and SA are in "g" in this example, while PID and RID are "rad"
demand_sample_ext.loc['Units', ['PFA', 'SA_1.13']] = 'g'
demand_sample_ext.loc['Units',['PID', 'RID']] = 'rad'

PAL.demand.load_sample(demand_sample_ext)

## 4.3 Damage

In [ ]:
# specify number of stories
PAL.stories = 1

# load component definitions
# cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)
PAL.asset.load_cmp_model({'marginals': cmp_marginals})

# generate sample
PAL.asset.generate_cmp_sample(sample_size)

# load the models into pelicun
PAL.damage.load_damage_model([
    additional_fragility_db,  # This is the extra fragility data we've just created
    'PelicunDefault/fragility_DB_FEMA_P58_2nd.csv' # and this is a table with the default P58 data    
])

# prescribe the damage process
dmg_process = {
    "1_collapse": {
        "DS1": "ALL_NA"
    },
    "2_excessiveRID": {
        "DS1": "irreparable_DS1"
    }
}

# calculate damages
PAL.damage.calculate(dmg_process=dmg_process)

## 4.4 Losses

In [ ]:
# create the loss map
drivers = [f'DMG-{cmp}' for cmp in cmp_marginals.index.unique()]
drivers = drivers[:-3]+drivers[-2:]

loss_models = cmp_marginals.index.unique().tolist()[:-3] +['replacement',]*2

loss_map = pd.DataFrame(loss_models, columns=['BldgRepair'], index=drivers)

# load the loss model
PAL.bldg_repair.load_model(
    [additional_consequences,
     "PelicunDefault/bldg_repair_DB_FEMA_P58_2nd.csv"], 
    loss_map)

# perform the calculation
PAL.bldg_repair.calculate()

# get the aggregate losses
agg_DF = PAL.bldg_repair.aggregate_losses()

agg_DF.describe([0.1, 0.5, 0.9])

In [ ]:
agg_DF

In [ ]:
from Run_pelicun_v3p1 import compile_demand_data

df_raw = compile_demand_data([sdr, pfa], ['SDR', 'PFA'])

In [ ]:
df_raw.head()

In [ ]:
#parameters
sample_size = 1000
delta_y = 0.0075
stripe = 3
PAL = Assessment({"PrintLog": False, "Seed": 415,})
raw_demands = compile_demand_data([sdr, pfa], ['SDR', 'PFA'])
baselineID = 's4_96x48'

In [ ]:
def loss_assessment(PAL, baseDir, baselineID, raw_demands, stripe, delta_y=0.0075, sample_size=1000):
    # prepare the demand input for pelicun
    stripe_demands = raw_demands.loc[str(stripe),:]

    # units - - - - - - - - - - - - - - - - - - - - - - - -  
    stripe_demands.insert(0, 'Units',"")
    stripe_demands.loc['PFA','Units'] = 'g'
    stripe_demands.loc['PID','Units'] = 'rad'


    # prepare a correlation matrix that represents perfect correlation
    ndims = stripe_demands.shape[0]
    demand_types = stripe_demands.index 

    perfect_CORR = pd.DataFrame(
        np.ones((ndims, ndims)),
        columns = demand_types,
        index = demand_types)

    # prepare additional fragility and consequence data ahead of time
    # cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)

    cmp_marginals = pd.read_csv(os.path.join(baseDir, *['BuildingInfo',
                                                        baselineID,
                                                        'ComponentsList',
                                                        'components_list_marginals.csv']), index_col=1)

    # add missing data to P58 damage model
    P58_data = PAL.get_default_data('fragility_DB_FEMA_P58_2nd')
    cmp_list = cmp_marginals.index.unique().values[:-3]

    # now take those components that are incomplete, and add the missing information
    additional_fragility_db = P58_data.loc[cmp_list,:].loc[P58_data.loc[cmp_list,'Incomplete'] == 1].sort_index()

    # prepare the extra damage models for collapse and irreparable damage
    additional_fragility_db.loc[
        'excessiveRID', [('Demand','Directional'),
                        ('Demand','Offset'),
                        ('Demand','Type'), 
                        ('Demand','Unit')]] = [1, 0, 'Residual Interstory Drift Ratio', 'rad']   

    additional_fragility_db.loc[
        'excessiveRID', [('LS1','Family'),
                        ('LS1','Theta_0'),
                        ('LS1','Theta_1')]] = ['lognormal', 0.01, 0.3]   

    additional_fragility_db.loc[
        'irreparable', [('Demand','Directional'),
                        ('Demand','Offset'),
                        ('Demand','Type'), 
                        ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   

    additional_fragility_db.loc[
        'irreparable', ('LS1','Theta_0')] = 1e10

    additional_fragility_db.loc[
        'collapse', [('Demand','Directional'),
                     ('Demand','Offset'),
                     ('Demand','Type'), 
                     ('Demand','Unit')]] = [1, 0, 'Peak Spectral Acceleration|1.13', 'g']   

    additional_fragility_db.loc[
        'collapse', [('LS1','Family'),
                     ('LS1','Theta_0'),
                     ('LS1','Theta_1')]] = ['lognormal', 1.35, 0.5]  

    # Now we can set the incomplete flag to 0 for these components
    additional_fragility_db['Incomplete'] = 0

    # create the additional consequence models
    additional_consequences = pd.DataFrame(
        columns = pd.MultiIndex.from_tuples([
            ('Incomplete',''), ('Quantity','Unit'), ('DV', 'Unit'), ('DS1', 'Theta_0')]),
        index=pd.MultiIndex.from_tuples([
            ('replacement','Cost'), ('replacement','Time')])
    )

    additional_consequences.loc[('replacement', 'Cost')] = [0, '1 EA', 'USD_2011', 21600000]
    additional_consequences.loc[('replacement', 'Time')] = [0, '1 EA', 'worker_day', 12500]

    # load the demand model
    PAL.demand.load_model({'marginals': stripe_demands,
                           'correlation': perfect_CORR})

    # generate samples
    PAL.demand.generate_sample({"SampleSize": sample_size})

    # add residual drift and Sa
    demand_sample = PAL.demand.save_sample()

    RID = PAL.demand.estimate_RID(demand_sample['PID'], {'yield_drift': delta_y})
    demand_sample_ext = pd.concat([demand_sample, RID], axis=1)

    Sa_vals = [0.158, 0.387, 0.615, 0.843, 1.071, 1.299, 1.528, 1.756]
    demand_sample_ext[('SA_1.13',0,1)] = Sa_vals[stripe-1]

    # add units to the data 
    demand_sample_ext.T.insert(0, 'Units',"")

    # PFA and SA are in "g" in this example, while PID and RID are "rad"
    demand_sample_ext.loc['Units', ['PFA', 'SA_1.13']] = 'g'
    demand_sample_ext.loc['Units',['PID', 'RID']] = 'rad'

    PAL.demand.load_sample(demand_sample_ext)

    # specify number of stories
    PAL.stories = 4

    # load component definitions
    # cmp_marginals = pd.read_csv('CMP_marginals.csv', index_col=0)
    PAL.asset.load_cmp_model({'marginals': cmp_marginals})

    # generate sample
    PAL.asset.generate_cmp_sample(sample_size)

    # load the models into pelicun
    PAL.damage.load_damage_model([
        additional_fragility_db,  # This is the extra fragility data we've just created
        'PelicunDefault/fragility_DB_FEMA_P58_2nd.csv' # and this is a table with the default P58 data    
    ])

    # prescribe the damage process
    dmg_process = {
        "1_collapse": {
            "DS1": "ALL_NA"
        },
        "2_excessiveRID": {
            "DS1": "irreparable_DS1"
        }
    }

    # calculate damages
    PAL.damage.calculate(dmg_process=dmg_process)

    # create the loss map
    drivers = [f'DMG-{cmp}' for cmp in cmp_marginals.index.unique()]
    drivers = drivers[:-3]+drivers[-2:]

    loss_models = cmp_marginals.index.unique().tolist()[:-3] +['replacement',]*2

    loss_map = pd.DataFrame(loss_models, columns=['BldgRepair'], index=drivers)

    # load the loss model
    PAL.bldg_repair.load_model(
        [additional_consequences,
         "PelicunDefault/bldg_repair_DB_FEMA_P58_2nd.csv"], 
        loss_map)

    # perform the calculation
    PAL.bldg_repair.calculate()

    # get the aggregate losses
    agg_DF = PAL.bldg_repair.aggregate_losses()
    return agg_DF

In [ ]:
a = loss_assessment(PAL, baseDir, baselineID, raw_demands, stripe, delta_y=0.0075, sample_size=1000)
a

In [ ]:
from Run_pelicun_v3p1 import pelicun_assessment1

b = pelicun_assessment1(PAL, baseDir, baselineID, raw_demands, stripe, delta_y=0.0075, sample_size=1000)

In [ ]:
b